In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import pandas_datareader as web 
plt.style.use('fivethirtyeight')

/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [ ]:
# Creating stock dataframe 
stock = web.DataReader('AAPL', data_source='yahoo', start = '2009-05-03', end = '2019-05-03')
stock = stock.reset_index()
stock

,Date,High,Low,Open,Close,Volume,Adj Close
0,2009-05-04,18.892857,18.240000,18.320000,18.867144,152339600.0,16.333237
1,2009-05-05,18.980000,18.731428,18.821428,18.958570,99563800.0,16.412386
2,2009-05-06,19.071428,18.602858,19.047142,18.928572,118384700.0,16.386417
3,2009-05-07,18.912857,18.271429,18.904285,18.437143,132944000.0,15.960989
4,2009-05-08,18.747143,18.037144,18.434286,18.455715,116991000.0,15.977063
...,...,...,...,...,...,...,...
2513,2019-04-29,205.970001,203.860001,204.399994,204.610001,22204700.0,201.421219
2514,2019-04-30,203.399994,199.110001,203.059998,200.669998,46534900.0,197.542618
2515,2019-05-01,215.309998,209.229996,209.880005,210.520004,64827300.0,207.239120
2516,2019-05-02,212.649994,208.130005,209.839996,209.149994,31996300.0,205.890457


In [ ]:
# visualizing data
trace = go.Scatter(x = stock['Date'], y = stock['Close'], marker = {'color':"#3f8fcc"})
layout = go.Layout(title = {'text':'AAPL 10Y <b>Close price</b>','x':0.5,'y':0.88,'font':{'color':'#404040'}},plot_bgcolor="white")
layout.xaxis.gridcolor = "#e3e3e3"
layout.yaxis.gridcolor = "#e3e3e3"
layout.xaxis.title.text = "03 June 2009 - 03 June 2019"
layout.yaxis.title.text = "AAPL close price"
fig = go.Figure(data = [trace], layout = layout)
fig.show()

In [ ]:
# constructing the 14 day rsi 
delta = stock['Close'].diff()
dUp, dDown = delta.copy(), delta.copy()
dUp[dUp < 0] = 0
dDown[dDown > 0] = 0
avg_gain = dUp.rolling(window = 14).mean()
avg_loss = dDown.rolling(window = 14).mean().abs()

rs = avg_gain / avg_loss 

rsi14 = 100 - (100/(1 + rs))

In [ ]:
# visualizing data
trace = go.Scatter(x = stock.Date, y=rsi14, marker={'color':'orange'}, name = "RSI 14")
layout = go.Layout(title = {'text':'AAPL 10Y <b>Relative Strength Index</b> 14 day trend','x':0.5,'y':0.88,'font':{'color':'#404040'}}, plot_bgcolor="white")
layout.xaxis.title.text = "03 June 2009 - 03 June 2019"
layout.xaxis.gridcolor = "#e3e3e3"
layout.yaxis.gridcolor = "#e3e3e3"
layout.yaxis.title.text = "$USD"
fig = go.Figure(data = [trace], layout = layout)
fig.show()


In [ ]:
# constructing dataframe 
df = pd.DataFrame()
df['close'] = stock.Close
df['rsi14'] = rsi14

In [ ]:
# returns buy/sell signals
def signal(data):
  flag = -1
  can_buy = True 
  can_sell = False
  buySigPrice = []
  sellSigPrice = []
  for i in range(len(data)):
    if data['rsi14'][i] > 70:
      if can_sell:
        sellSigPrice.append(data['close'][i])
        buySigPrice.append(np.nan)
        can_sell, can_buy = False, True
      else:
        sellSigPrice.append(np.nan)
        buySigPrice.append(np.nan)
    elif data['rsi14'][i] < 30:
      if (can_buy):
        buySigPrice.append(data['close'][i])
        sellSigPrice.append(np.nan)
        can_buy, can_sell = False, True 
      else:
        sellSigPrice.append(np.nan)
        buySigPrice.append(np.nan)
    else:
        buySigPrice.append(np.nan)
        sellSigPrice.append(np.nan)

  return (np.array(buySigPrice), np.array(sellSigPrice))



In [ ]:
# store buy/sell price
decision = signal(df)
df['buy signal price'] = decision[0]
df['sell signal price'] = decision[1]
df.reset_index()

,index,close,rsi14,buy signal price,sell signal price
0,0,18.867144,NaN,NaN,NaN
1,1,18.958570,NaN,NaN,NaN
2,2,18.928572,NaN,NaN,NaN
3,3,18.437143,NaN,NaN,NaN
4,4,18.455715,NaN,NaN,NaN
...,...,...,...,...,...
2513,2513,204.610001,64.482970,NaN,NaN
2514,2514,200.669998,53.093598,NaN,NaN
2515,2515,210.520004,67.908845,NaN,NaN
2516,2516,209.149994,68.653973,NaN,NaN


In [ ]:
# Visualizing buy sell signals
trace = go.Scatter(x = stock.Date, y = df['close'], name = 'AAPL close price',  marker = {'color':"#3f8fcc"}, opacity = 0.5)
buy = go.Scatter(x = stock.Date, y = df['buy signal price'], marker = {'color':'green', 'symbol':'triangle-up','size':7}, name = "Buy", mode = 'markers')
sell = go.Scatter(x = stock.Date, y = df['sell signal price'], marker = {'color':'red', 'symbol':'triangle-down','size':7}, name = "Sell", mode = 'markers')
layout = go.Layout(title = {'text':'AAPL 10Y <b>Buy-Sell Signals</b>','x':0.5,'y':0.88,'font':{'color':'#404040'}}, plot_bgcolor = "white")
layout.xaxis.title.text = "03 June 2009 - 03 June 2019"
layout.xaxis.gridcolor = "#e3e3e3"
layout.yaxis.gridcolor = "#e3e3e3"
layout.yaxis.title.text = "AAPL close price"
fig = go.Figure(data = [trace,buy,sell], layout = layout)
fig.show()

# Visualizing buy-sell signal lines
trace = go.Scatter(x = stock.Date, y=rsi14, marker={'color':'orange'}, name = "RSI 14")
sell = go.Scatter(x = stock.Date, y=[70]*len(df), opacity = 0.40, marker={'color':'red'}, name = "sell signal line")
buy = go.Scatter(x = stock.Date, y=[30]*len(df), opacity = 0.40, marker={'color':'green'}, name = "buy signal line")
layout = go.Layout(title = {'text':'AAPL 10Y <b>Relative Strength Index</b> 14 day trend','x':0.5,'y':0.88,'font':{'color':'#404040'}}, plot_bgcolor="white")
layout.xaxis.title.text = "03 June 2009 - 03 June 2019"
layout.xaxis.gridcolor = "#e3e3e3"
layout.yaxis.gridcolor = "#e3e3e3"
layout.yaxis.title.text = "$USD"
fig = go.Figure(data = [trace, buy, sell], layout = layout)
fig.show()


In [ ]:
# evaluating total advantage: 
adv = df['sell signal price'].copy().fillna(0).sum() - df['buy signal price'].copy().fillna(0).sum() 
print("AAPL stats : ")
print("----------------------------------------------------------------------")
print("Min : " + str(df['close'].min()))
print("Max : " + str(df['close'].max()))
print("Trading time : 10Y (June 03 2009  - June 03 2019) " )
print("----------------------------------------------------------------------")
print('Total Profit using rsi14 stratergy :' + str(adv))

AAPL stats : 
----------------------------------------------------------------------
Min : 17.06999969482422
Max : 232.07000732421875
Trading time : 10Y (June 03 2009  - June 03 2019) 
----------------------------------------------------------------------
Total Profit using rsi14 stratergy :89.11573219299316


In [ ]:
# Accounting for profits and losses 
buy = [x for x in df['buy signal price']  if str(x) != 'nan']
sell = [x for x in df['sell signal price']  if str(x) != 'nan']
profit = []
loss = []
for i in range(len(buy)):
  difference = sell[i] - buy[i]
  if  difference > 0:
    profit.append(difference)
  elif difference < 0:
    loss.append(difference)

In [ ]:
# Analysis of RSI trading stratergy
fig = go.Figure(go.Indicator(
    mode = "number+gauge",
    gauge = {'shape': "bullet",
             'bar':{'color':'green'}},
    value = df['sell signal price'].mean(),
    domain = {'x': [0.20, 1], 'y': [0.08, 0.25]},
    title = {'text': "Average value per trade"}))

fig.add_trace(go.Indicator(
    mode = "number+gauge",
    gauge = {'shape': "bullet",
             'bar':{'color':'red'}},
    value = df['buy signal price'].mean(),
    domain = {'x': [0.20, 1], 'y': [0.4, 0.6]},
    title = {'text': "Average cost per trade"}))

fig.add_trace(go.Indicator(
    mode = "number+gauge+delta",
    gauge = {'shape': "bullet",
             'bar':{'color':'orange'}},
    delta = {'reference': stock['Close'][len(stock) - 1] - adv},
    value = stock['Close'][len(stock) - 1],
    domain = {'x': [0.20, 1], 'y': [0.7, 0.9]},
    title = {'text': "Current value"}))

fig.show()


In [ ]:
# Use `hole` to create a donut-like pie chart
labels = ['Profit','Loss']
values = []
fig = go.Figure(data=[go.Pie(labels=labels, values=[len(profit),len(loss)], hole=.3,title={'text':"Share of P/L in all trades",})])
fig.show()


In [ ]:
# Visualizing using a box plot

trace_buy = go.Box(y = profit, name = "Profit")
trace_sell = go.Box(y = np.array(loss)*-1, name = "Loss")
layout = go.Layout(title={'text':"Visualizing <b>profit and loss distribution</b>", 'x':0.5,'y':1, 'font':{'color':'#404040'}}, plot_bgcolor="white")
layout.xaxis.gridcolor = "#e3e3e3"
layout.yaxis.gridcolor = "#e3e3e3"
fig = go.Figure(data = [trace_buy, trace_sell], layout = layout)
fig.show()


In [ ]:
#using bubble plot 
x_profit = [i for i in range(len(profit))]
x_loss = np.array([i for i in range(len(profit), len(profit)+len(loss))])
trace1 = go.Scatter(x = x_profit, y = np.array(profit), marker={'size':np.array(profit) * 5}, mode="markers", name="profit")
trace2 = go.Scatter(x = x_loss, y = np.array(loss)*-1, marker={'size':np.array(loss)* -5}, mode="markers", name="loss")
layout = go.Layout(title={'text':"Visualizing <b>profit and loss distribution</b>", 'x':0.5,'y':0.92, 'font':{'color':'#404040'}}, plot_bgcolor="white")
layout.xaxis.gridcolor = "#e3e3e3"
layout.yaxis.gridcolor = "#e3e3e3"
fig = go.Figure(data = [trace1, trace2], layout = layout)
fig.show()